# The Field Effect: How MOSFETs Work

**Module 04 — MOSFETs | Notebook 00**

---

The BJT was revolutionary — but it has a fundamental limitation: it needs *continuous base current* to stay on. Every milliamp flowing into the base is wasted energy that does no useful work in your load. The MOSFET flips the script entirely: it's controlled by *voltage*, not current. The gate is physically insulated from the channel by a thin oxide layer, so essentially **zero current flows into the gate at DC**.

This single difference — voltage control vs. current control — is why MOSFETs dominate modern electronics.

## Concept — Voltage-Controlled vs. Current-Controlled Devices

| Feature | BJT | MOSFET |
|---|---|---|
| Control input | Base *current* (I_B) | Gate *voltage* (V_GS) |
| Input impedance | Low (~kΩ) | Extremely high (~10¹² Ω) |
| DC gate/base current | mA range | Essentially zero |
| Main equation | I_C = β × I_B | I_D = f(V_GS, V_DS) |
| Switching energy | Moderate | Low (just charge/discharge gate capacitance) |

A BJT is like a water valve that requires you to *push* on it continuously. A MOSFET is like a valve with a latch — you push it open, and it stays open with no effort until you unlatch it.

## Concept — MOSFET Structure: Gate, Drain, Source

```
        Enhancement-mode N-Channel MOSFET Cross-Section

            Gate (G)
              │
         ┌────┴────┐
         │  Metal   │
         ├─────────┤
         │  SiO₂    │  ← Gate oxide (insulator!)
         ├─────────┤
    ┌────┤ P-type   ├────┐
    │N+  │  body    │ N+ │
    │    │          │    │
    Source(S)      Drain(D)
```

**Key structural points:**
- The **gate** is a metal (or polysilicon) electrode separated from the silicon by a very thin **oxide** (SiO₂) — this is the 'O' in MOSFET (Metal-Oxide-Semiconductor)
- **Source** and **Drain** are heavily doped N+ regions in a P-type body
- With no gate voltage, there is *no conducting path* between source and drain — the two N+ regions are isolated by the P-type body
- The oxide is incredibly thin — typically 5-50 nm. This is why MOSFETs are ESD sensitive!

## Concept — Enhancement-Mode N-Channel: Creating the Channel

When we apply a positive voltage to the gate (relative to the source):

1. The positive charge on the gate creates an **electric field** through the oxide
2. This field *repels* holes (positive carriers) away from the surface of the P-type body
3. It simultaneously *attracts* electrons to the surface
4. When enough electrons accumulate, they form a thin **inversion layer** — an N-type channel connecting source to drain
5. Current can now flow from drain to source through this channel

**The channel is created by the electric field — the "field effect."**

The minimum V_GS needed to form this channel is the **threshold voltage, V_GS(th)**.

## The Material Science Why

### What's physically happening in the silicon

The P-type silicon body has an abundance of holes (missing electrons). Near the surface under the gate:

1. **Accumulation** (V_GS < 0): Negative gate voltage attracts holes *toward* the surface. No channel.

2. **Depletion** (0 < V_GS < V_th): Small positive gate voltage pushes holes *away* from the surface, creating a depletion region. Still no conducting channel.

3. **Inversion** (V_GS > V_th): Enough positive voltage on the gate has pushed all holes away and attracted minority carrier electrons. The surface has *inverted* from P-type to effectively N-type. **This is the channel.** It's only a few nanometers thick — a 2D sheet of electrons.

The gate oxide is the hero here. SiO₂ is one of nature's best insulators. Silicon naturally grows a high-quality oxide when exposed to oxygen — this lucky accident of chemistry is a major reason silicon won the semiconductor race over germanium (which forms a lousy oxide).

The oxide must be *thin enough* for the electric field to reach the silicon surface, but *thick enough* to not break down. Modern CPUs use oxide layers just a few atoms thick.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# MOSFET output characteristics: I_D vs V_DS for different V_GS
# Using simplified square-law model:
#   Linear region: I_D = K * [2*(V_GS - V_th)*V_DS - V_DS^2]
#   Saturation:    I_D = K * (V_GS - V_th)^2

V_th = 2.0       # threshold voltage (V)
K = 0.5           # transconductance parameter (A/V^2) — simplified
lambda_val = 0.02 # channel-length modulation

VDS = np.linspace(0, 10, 500)
VGS_values = [2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(VGS_values)))

fig, ax = plt.subplots(figsize=(10, 6))

for VGS, color in zip(VGS_values, colors):
    if VGS <= V_th:
        ID = np.zeros_like(VDS)
    else:
        V_ov = VGS - V_th  # overdrive voltage
        V_DS_sat = V_ov    # saturation voltage
        
        ID = np.where(
            VDS < V_DS_sat,
            # Linear (ohmic) region
            K * (2 * V_ov * VDS - VDS**2),
            # Saturation region (with channel-length modulation)
            K * V_ov**2 * (1 + lambda_val * (VDS - V_DS_sat))
        )
    
    ax.plot(VDS, ID, color=color, linewidth=2, label=f'V_GS = {VGS:.1f} V')
    # Mark the boundary between linear and saturation
    if VGS > V_th:
        V_DS_sat = VGS - V_th
        ID_sat = K * (VGS - V_th)**2
        ax.plot(V_DS_sat, ID_sat, 'o', color=color, markersize=6)

# Draw the locus of V_DS(sat) points
vgs_fine = np.linspace(V_th, 5.0, 100)
vds_sat_curve = vgs_fine - V_th
id_sat_curve = K * (vgs_fine - V_th)**2
ax.plot(vds_sat_curve, id_sat_curve, 'k--', linewidth=1, alpha=0.5, label='Linear ↔ Saturation boundary')

ax.set_xlabel('V_DS (V)', fontsize=12)
ax.set_ylabel('I_D (A)', fontsize=12)
ax.set_title('MOSFET Output Characteristics — I_D vs V_DS', fontsize=14)
ax.legend(fontsize=10)
ax.set_xlim(0, 10)
ax.set_ylim(0, 5)
ax.grid(True, alpha=0.3)
ax.annotate('Linear\n(Ohmic)\nRegion', xy=(0.5, 1.5), fontsize=11, ha='center',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', alpha=0.8))
ax.annotate('Saturation\nRegion', xy=(6, 1.5), fontsize=11, ha='center',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

print("Each curve shows I_D vs V_DS for a fixed V_GS.")
print("Dots mark where each curve transitions from linear to saturation.")
print(f"Below V_GS = {V_th} V (threshold), the MOSFET is OFF — no curves appear.")

In [ ]:
# Transfer characteristic: I_D vs V_GS (at fixed V_DS in saturation)

V_th = 2.0
K = 0.5
VGS = np.linspace(0, 6, 500)

# In saturation: I_D = K * (V_GS - V_th)^2 for V_GS > V_th
ID = np.where(VGS > V_th, K * (VGS - V_th)**2, 0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
ax1.plot(VGS, ID, 'b-', linewidth=2.5)
ax1.axvline(x=V_th, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label=f'V_GS(th) = {V_th} V')
ax1.fill_between(VGS, 0, ID, where=(VGS < V_th), color='lightcoral', alpha=0.2, label='Cutoff region')
ax1.fill_between(VGS, 0, ID, where=(VGS >= V_th), color='lightgreen', alpha=0.2, label='Channel conducting')
ax1.set_xlabel('V_GS (V)', fontsize=12)
ax1.set_ylabel('I_D (A)', fontsize=12)
ax1.set_title('Transfer Characteristic (Linear Scale)', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 6)
ax1.set_ylim(0, 9)

# Square root scale to show linearity
sqrt_ID = np.sqrt(ID)
ax2.plot(VGS, sqrt_ID, 'b-', linewidth=2.5)
ax2.axvline(x=V_th, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label=f'V_GS(th) = {V_th} V')
# Extrapolation line
vgs_extrap = np.linspace(V_th - 0.5, 6, 100)
sqrt_extrap = np.sqrt(K) * np.maximum(vgs_extrap - V_th, 0)
ax2.plot(vgs_extrap, sqrt_extrap, 'r:', linewidth=1.5, alpha=0.5, label='Linear extrapolation')
ax2.set_xlabel('V_GS (V)', fontsize=12)
ax2.set_ylabel('√I_D (√A)', fontsize=12)
ax2.set_title('√I_D vs V_GS — Extrapolate to Find V_th', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.annotate('V_th', xy=(V_th, 0), xytext=(V_th + 0.3, 0.5),
            fontsize=12, color='red',
            arrowprops=dict(arrowstyle='->', color='red'))

plt.tight_layout()
plt.show()

print("Left: The classic 'square-law' curve. Below V_th, the MOSFET is OFF.")
print("Right: Plot sqrt(I_D) vs V_GS — it becomes a straight line!")
print("       Extrapolating that line to the x-axis gives you V_th.")

## Concept — MOSFET Regions of Operation

An N-channel enhancement MOSFET has three operating regions:

### 1. Cutoff (V_GS < V_th)
- No channel exists — the MOSFET is **OFF**
- I_D ≈ 0 (only tiny leakage current flows)
- The switch is open

### 2. Linear / Ohmic / Triode (V_GS > V_th AND V_DS < V_GS - V_th)
- Channel exists and is *not pinched off*
- MOSFET behaves like a **voltage-controlled resistor**
- I_D ≈ K × [2(V_GS - V_th) × V_DS - V_DS²]
- For small V_DS: I_D ≈ (1/R_DS(on)) × V_DS — a simple resistor!
- **This is where we operate MOSFETs as switches** — V_DS is small, so power loss is low

### 3. Saturation (V_GS > V_th AND V_DS ≥ V_GS - V_th)
- Channel is *pinched off* at the drain end
- I_D ≈ K × (V_GS - V_th)² — current depends mainly on V_GS, not V_DS
- MOSFET behaves like a **voltage-controlled current source**
- **This is where we operate MOSFETs as amplifiers**

> **Critical insight:** When using a MOSFET as a switch, you want it fully in the *linear region* (V_GS >> V_th, V_DS very small). When using it as an amplifier, you want it in *saturation*. Confusingly, BJT "saturation" means the switch is fully ON, but MOSFET "saturation" means constant-current mode!

In [ ]:
# Interactive widget: adjust VGS and VDS, see operating point
# Falls back gracefully if ipywidgets is not available

import numpy as np
import matplotlib.pyplot as plt

try:
    from ipywidgets import interact, FloatSlider
    HAS_WIDGETS = True
except ImportError:
    HAS_WIDGETS = False
    print("ipywidgets not installed — showing static example instead.")

V_th = 2.0
K = 0.5

def plot_operating_point(VGS=4.0, VDS=3.0):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    vds_range = np.linspace(0, 10, 500)
    
    # Plot family of curves
    for vgs_val in [2.5, 3.0, 3.5, 4.0, 4.5, 5.0]:
        if vgs_val <= V_th:
            continue
        v_ov = vgs_val - V_th
        v_ds_sat = v_ov
        id_curve = np.where(
            vds_range < v_ds_sat,
            K * (2 * v_ov * vds_range - vds_range**2),
            K * v_ov**2 * (1 + 0.02 * (vds_range - v_ds_sat))
        )
        alpha = 1.0 if abs(vgs_val - VGS) < 0.01 else 0.3
        lw = 2.5 if abs(vgs_val - VGS) < 0.01 else 1.0
        ax.plot(vds_range, id_curve, linewidth=lw, alpha=alpha, 
                label=f'V_GS={vgs_val:.1f}V' if abs(vgs_val - VGS) < 0.01 else '')
    
    # Calculate operating point
    if VGS <= V_th:
        ID_point = 0
        region = "CUTOFF"
        region_color = 'red'
    else:
        v_ov = VGS - V_th
        v_ds_sat = v_ov
        if VDS < v_ds_sat:
            ID_point = K * (2 * v_ov * VDS - VDS**2)
            region = "LINEAR (Ohmic)"
            region_color = 'green'
        else:
            ID_point = K * v_ov**2 * (1 + 0.02 * (VDS - v_ds_sat))
            region = "SATURATION"
            region_color = 'blue'
    
    ax.plot(VDS, ID_point, 'ro', markersize=15, zorder=5)
    ax.annotate(f'Operating Point\nV_DS={VDS:.1f}V, I_D={ID_point:.2f}A\nRegion: {region}',
               xy=(VDS, ID_point), xytext=(VDS + 1, ID_point + 0.5),
               fontsize=11, color=region_color, fontweight='bold',
               arrowprops=dict(arrowstyle='->', color=region_color),
               bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor=region_color))
    
    ax.set_xlabel('V_DS (V)', fontsize=12)
    ax.set_ylabel('I_D (A)', fontsize=12)
    ax.set_title(f'MOSFET Operating Point — V_GS = {VGS:.1f} V, V_DS = {VDS:.1f} V', fontsize=13)
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 6)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

if HAS_WIDGETS:
    interact(plot_operating_point,
             VGS=FloatSlider(min=0, max=5.0, step=0.5, value=4.0, description='V_GS (V):'),
             VDS=FloatSlider(min=0, max=10.0, step=0.5, value=3.0, description='V_DS (V):'))
else:
    plot_operating_point(VGS=4.0, VDS=3.0)

## Concept — Gate Capacitance: The Gate Is a Capacitor

The gate oxide creates a **parallel-plate capacitor** between the gate electrode and the channel:

```
    Gate metal  ─────────────  ← Plate 1
    SiO₂ oxide  ─ ─ ─ ─ ─ ─  ← Dielectric
    Silicon      ─────────────  ← Plate 2
```

**Consequences:**
- **No DC current flows** through the gate — the oxide is an insulator. Input impedance is ~10¹² Ω.
- **AC current does flow** — you must charge/discharge this capacitance to change the gate voltage.
- The **gate charge (Q_g)** determines how fast you can switch the MOSFET. More charge = slower switching or beefier gate driver needed.
- Typical gate capacitance: 100 pF to 10 nF for power MOSFETs.

This is why MOSFETs are easy to drive in DC (no current needed) but require attention at high switching frequencies (must supply transient current to charge the gate).

## Concept — The Body Diode

Look at the MOSFET structure again — the P-type body between the N+ source and N+ drain forms PN junctions. The source is usually connected to the body internally, which creates a **parasitic diode** from source to drain:

```
    Drain (D)
      │
      │  N+
      │──┤├──│  ← PN junction (body diode)
      │  P-body
      │  N+
      │
    Source (S)
```

The body diode conducts when V_DS goes negative (for N-channel). This is:
- **Useful** in motor drive circuits (freewheeling diode is built-in!)
- **Problematic** in some switching circuits (the body diode is slow compared to dedicated fast diodes)
- **Always present** — you can't get rid of it; it's part of the structure

The datasheet specifies body diode parameters: forward voltage (V_SD), current rating (I_S), and reverse recovery time (t_rr).

In [ ]:
# Draw N-channel and P-channel MOSFET symbols

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyArrowPatch

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

def draw_nmos(ax):
    """Draw N-channel enhancement MOSFET symbol."""
    ax.set_xlim(-2, 4)
    ax.set_ylim(-2, 4)
    ax.set_aspect('equal')
    ax.axis('off')
    
    # Gate line (vertical bar)
    ax.plot([0.5, 0.5], [0.5, 2.5], 'k-', linewidth=3)
    # Gate terminal
    ax.plot([-1, 0.5], [1.5, 1.5], 'k-', linewidth=2)
    
    # Channel segments (3 dashes for enhancement mode)
    ax.plot([1.0, 1.0], [2.1, 2.5], 'k-', linewidth=3)
    ax.plot([1.0, 1.0], [1.3, 1.7], 'k-', linewidth=3)
    ax.plot([1.0, 1.0], [0.5, 0.9], 'k-', linewidth=3)
    
    # Drain connection (top)
    ax.plot([1.0, 1.0], [2.3, 3.5], 'k-', linewidth=2)
    ax.plot([1.0, 2.5], [3.5, 3.5], 'k-', linewidth=2)
    
    # Source connection (bottom)
    ax.plot([1.0, 1.0], [0.7, -0.5], 'k-', linewidth=2)
    ax.plot([1.0, 2.5], [-0.5, -0.5], 'k-', linewidth=2)
    
    # Middle connection to body (with arrow pointing IN for N-channel)
    ax.plot([1.0, 2.5], [1.5, 1.5], 'k-', linewidth=2)
    # Arrow on middle segment pointing inward (toward channel)
    ax.annotate('', xy=(1.0, 1.5), xytext=(1.5, 1.5),
               arrowprops=dict(arrowstyle='->', color='black', lw=2))
    
    # Body diode (from S to D)
    ax.plot([2.5, 2.5], [-0.5, 3.5], 'k-', linewidth=2)
    # Diode symbol on the vertical line
    ax.plot([2.2, 2.5, 2.8], [1.0, 1.5, 1.0], 'k-', linewidth=1.5)  # triangle
    ax.plot([2.2, 2.8], [1.5, 1.5], 'k-', linewidth=1.5)  # bar
    
    # Labels
    ax.text(-1.2, 1.5, 'G', fontsize=14, fontweight='bold', ha='center', va='center')
    ax.text(2.5, 3.8, 'D', fontsize=14, fontweight='bold', ha='center', va='center')
    ax.text(2.5, -0.9, 'S', fontsize=14, fontweight='bold', ha='center', va='center')
    ax.set_title('N-Channel Enhancement MOSFET', fontsize=13, fontweight='bold')

def draw_pmos(ax):
    """Draw P-channel enhancement MOSFET symbol."""
    ax.set_xlim(-2, 4)
    ax.set_ylim(-2, 4)
    ax.set_aspect('equal')
    ax.axis('off')
    
    # Gate line (vertical bar)
    ax.plot([0.5, 0.5], [0.5, 2.5], 'k-', linewidth=3)
    # Gate terminal with bubble (circle for inversion)
    ax.plot([-1, 0.1], [1.5, 1.5], 'k-', linewidth=2)
    circle = plt.Circle((0.3, 1.5), 0.15, fill=False, color='black', linewidth=2)
    ax.add_patch(circle)
    
    # Channel segments (3 dashes for enhancement mode)
    ax.plot([1.0, 1.0], [2.1, 2.5], 'k-', linewidth=3)
    ax.plot([1.0, 1.0], [1.3, 1.7], 'k-', linewidth=3)
    ax.plot([1.0, 1.0], [0.5, 0.9], 'k-', linewidth=3)
    
    # Source connection (top — P-channel source is at the positive rail)
    ax.plot([1.0, 1.0], [2.3, 3.5], 'k-', linewidth=2)
    ax.plot([1.0, 2.5], [3.5, 3.5], 'k-', linewidth=2)
    
    # Drain connection (bottom)
    ax.plot([1.0, 1.0], [0.7, -0.5], 'k-', linewidth=2)
    ax.plot([1.0, 2.5], [-0.5, -0.5], 'k-', linewidth=2)
    
    # Middle connection to body (with arrow pointing OUT for P-channel)
    ax.plot([1.0, 2.5], [1.5, 1.5], 'k-', linewidth=2)
    ax.annotate('', xy=(1.5, 1.5), xytext=(1.0, 1.5),
               arrowprops=dict(arrowstyle='->', color='black', lw=2))
    
    # Body diode (from D to S — reversed for P-channel)
    ax.plot([2.5, 2.5], [-0.5, 3.5], 'k-', linewidth=2)
    ax.plot([2.2, 2.5, 2.8], [1.5, 1.0, 1.5], 'k-', linewidth=1.5)  # triangle (flipped)
    ax.plot([2.2, 2.8], [1.0, 1.0], 'k-', linewidth=1.5)  # bar
    
    # Labels
    ax.text(-1.2, 1.5, 'G', fontsize=14, fontweight='bold', ha='center', va='center')
    ax.text(2.5, 3.8, 'S', fontsize=14, fontweight='bold', ha='center', va='center')
    ax.text(2.5, -0.9, 'D', fontsize=14, fontweight='bold', ha='center', va='center')
    ax.set_title('P-Channel Enhancement MOSFET', fontsize=13, fontweight='bold')

draw_nmos(ax1)
draw_pmos(ax2)

plt.tight_layout()
plt.show()

print("N-Channel: Arrow points IN. Apply positive V_GS to turn ON.")
print("P-Channel: Arrow points OUT + bubble on gate. Apply negative V_GS (pull gate LOW) to turn ON.")
print("Both show the body diode — it's always there in a power MOSFET.")

## Concept — P-Channel MOSFETs

A P-channel MOSFET is the complement of the N-channel:

| Parameter | N-Channel | P-Channel |
|---|---|---|
| Channel carriers | Electrons | Holes |
| Turn on when | V_GS > V_th (positive) | V_GS < -V_th (negative) |
| In practice | Pull gate HIGH | Pull gate LOW |
| Current flows | Drain → Source | Source → Drain |
| Typical use | Low-side switch | High-side switch |
| Performance | Better (electrons are faster) | Worse (holes are slower) |

**Why P-channel for high-side switching?**

```
    V+ ────┤S        V+ ──────────────
           │ PMOS              │
    GND ──│G               ┌──┤D
           │               │  │ NMOS
           ┤D         GND──│──│G ???
           │               │  ┤S
         [Load]          [Load]│
           │               │
          GND             GND
    P-channel: Easy!    N-channel: Need V_GS > V_th
    Gate to GND turns     above V+... problem!
    it fully ON.          (needs charge pump or
                           bootstrap circuit)
```

P-channel MOSFETs are simpler for high-side switching because pulling the gate to ground gives you the full supply voltage as V_GS.

## Concept — Depletion-Mode vs. Enhancement-Mode

Everything above describes **enhancement-mode** MOSFETs — the kind you'll use 99% of the time. They are *normally OFF* and you enhance (create) the channel by applying gate voltage.

**Depletion-mode** MOSFETs are *normally ON* — they have a built-in channel that exists with V_GS = 0. You apply gate voltage to *deplete* (remove) the channel and turn the device OFF.

| Mode | V_GS = 0 | To turn ON | To turn OFF |
|---|---|---|---|
| Enhancement | OFF | Apply V_GS > V_th | Remove gate voltage |
| Depletion | ON | Already on (or enhance further) | Apply negative V_GS |

Depletion-mode MOSFETs are rare in everyday circuits but show up in:
- Current sources
- RF circuits
- Some power supply start-up circuits

**For this course, when we say "MOSFET" we mean enhancement-mode unless stated otherwise.**

## Experiment — Observing the Field Effect (Conceptual)

We'll do hands-on MOSFET experiments in the next notebook. For now, here's the key experiment in concept:

```
         +12V
          │
        [LED]
          │
          ├───── Drain (D)
          │
          │     2N7000
          │     (N-channel MOSFET)
          │
    Pot ──├───── Gate (G)
    0-5V  │
          ├───── Source (S)
          │
         GND
```

**What you'd observe:**
- Slowly turn up the potentiometer from 0V
- Nothing happens until V_GS reaches ~2V (the threshold)
- LED suddenly begins to glow
- Brightness increases rapidly as V_GS increases further
- **Touch your finger to the gate pin** (with source grounded) — your body's static charge (~mV to volts) can turn it on! This demonstrates both the field effect and ESD sensitivity.

> **Warning:** Never leave a MOSFET gate floating in a real circuit! The extreme input impedance means stray charges can put any voltage on the gate, potentially turning it on unexpectedly or exceeding V_GS max and destroying the oxide.

## Simulation — Build It in Falstad

Build a basic MOSFET switching circuit to see the field effect in action.

**What to build:**

1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Right-click → **Draw** → **Active Components** → **Add MOSFET (n-channel)** — place it in the center
3. Connect a DC voltage source (5V) through a 470Ω resistor to the **drain** (top terminal)
4. Connect the **source** (bottom terminal) to ground
5. Connect a second voltage source (adjustable, start at 0V) directly to the **gate** (left terminal)
6. Add ground connections

**What to observe:**
- Set the gate voltage to 0V: no drain current — the MOSFET is off
- Slowly increase the gate voltage: at some point (the threshold voltage, V_GS(th)), drain current begins to flow
- Keep increasing: drain current grows. Hover over the drain resistor to see the current value.
- **Critical observation:** hover over the gate lead — there is essentially **zero gate current**. This is the fundamental difference from a BJT: the MOSFET is voltage-controlled, not current-controlled.

**What to try:**
- Try gate voltages of 0V, 2V, 3V, 4V, 5V and note the drain current at each — you'll see the transfer characteristic
- Right-click the MOSFET → **View in New Scope** to watch V_DS and I_D change as you adjust V_GS
- Replace with a P-channel MOSFET (right-click → **Draw** → **Active Components** → **Add MOSFET (p-channel)**) and reverse the supply — observe the same principle with opposite polarities

## Datasheet Connection

Every concept in this notebook maps directly to MOSFET datasheets:

| Concept | Datasheet Parameter | Where to Find It |
|---|---|---|
| Threshold voltage | V_GS(th) | Electrical Characteristics table |
| Gate is a capacitor | C_iss, Q_g | Gate charge / capacitance section |
| Linear region resistance | R_DS(on) | Electrical Characteristics (at specific V_GS) |
| Maximum gate voltage | V_GS (abs max) | Absolute Maximum Ratings (typically ±20V) |
| Body diode | V_SD, I_S, t_rr | Body diode section |
| Output curves | I_D vs V_DS graph | Characteristic Curves section |
| Transfer curve | I_D vs V_GS graph | Characteristic Curves section |

We'll do a deep-dive datasheet walkthrough for the IRFZ44N in Notebook 03.

In [ ]:
# Summary visualization: MOSFET regions on the VGS-VDS plane

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(10, 7))

V_th = 2.0

# VGS axis (y), VDS axis (x)
vds = np.linspace(0, 10, 300)

# Cutoff region: VGS < Vth (below threshold line)
ax.fill_between([0, 10], [0, 0], [V_th, V_th], color='lightcoral', alpha=0.3)

# Linear region: VGS > Vth AND VDS < VGS - Vth → VGS > VDS + Vth
vgs_boundary = vds + V_th
ax.fill_between(vds, V_th, np.minimum(vgs_boundary, 8), 
                where=(vgs_boundary > V_th), color='lightgreen', alpha=0.3)

# Saturation region: VGS > Vth AND VDS >= VGS - Vth → VGS <= VDS + Vth
ax.fill_between(vds, np.minimum(vgs_boundary, 8), 8, 
                where=(vds <= 6), color='lightblue', alpha=0.3)
ax.fill_between(vds, V_th, 8,
                where=(vds > 6), color='lightblue', alpha=0.3)

# Boundary line: VDS = VGS - Vth → VGS = VDS + Vth
vds_line = np.linspace(0, 6, 100)
vgs_line = vds_line + V_th
ax.plot(vds_line, vgs_line, 'k--', linewidth=2, label='V_DS = V_GS - V_th')

# Threshold line
ax.axhline(y=V_th, color='red', linestyle=':', linewidth=2, label=f'V_GS(th) = {V_th} V')

# Labels
ax.text(5, 1.0, 'CUTOFF\n(OFF)', fontsize=14, ha='center', va='center',
        fontweight='bold', color='darkred')
ax.text(1.0, 5.5, 'LINEAR\n(Switch ON)', fontsize=14, ha='center', va='center',
        fontweight='bold', color='darkgreen')
ax.text(7, 5.5, 'SATURATION\n(Amplifier)', fontsize=14, ha='center', va='center',
        fontweight='bold', color='darkblue')

ax.set_xlabel('V_DS (V)', fontsize=13)
ax.set_ylabel('V_GS (V)', fontsize=13)
ax.set_title('MOSFET Operating Regions on the V_GS – V_DS Plane', fontsize=14)
ax.legend(fontsize=11, loc='lower right')
ax.set_xlim(0, 10)
ax.set_ylim(0, 8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("For SWITCHING: operate in the LINEAR region (low V_DS, high V_GS).")
print("For AMPLIFYING: operate in the SATURATION region.")
print("Below V_th: device is OFF regardless of V_DS.")

## Checkpoint Questions

Test your understanding before moving on:

1. **Why does a MOSFET gate draw essentially zero DC current?** What physical structure causes this?

2. **What is the threshold voltage V_GS(th)?** What physically happens in the silicon at this voltage?

3. **A MOSFET is being used as a switch.** Which region should it operate in — linear or saturation? Why?

4. **You're designing a circuit and need a high-side switch** (load connected between switch and ground, switch connected to V+). Would you choose an N-channel or P-channel MOSFET? Why?

5. **Why is it dangerous to leave a MOSFET gate pin floating** (not connected to anything)? What could happen?

6. **What is the body diode?** In an N-channel MOSFET, which direction does it conduct?

7. **Compare BJT and MOSFET switching:** The BJT needs continuous base current to stay on. What does the MOSFET need to stay on? (Trick question — think carefully!)

8. **Why did silicon win over germanium** for making MOSFETs, from a material science perspective?

---

*Answers:*
1. *The gate oxide (SiO₂) is an insulator. The gate is one plate of a capacitor — no DC path exists.*
2. *The minimum V_GS to create an inversion layer (conducting channel) at the silicon surface.*
3. *Linear (ohmic) region — V_DS is very small, so power dissipation (I² × R_DS(on)) is minimized.*
4. *P-channel — pulling the gate to ground gives full V_GS without needing voltage above V+.*
5. *The extreme input impedance means stray charge can put any voltage on the gate — turning it on unexpectedly or exceeding V_GS(max) and destroying the oxide.*
6. *A parasitic diode formed by the body-drain PN junction. In N-channel, it conducts from source to drain (when V_DS goes negative).*
7. *Just the gate voltage — but it needs NO current to maintain that voltage (just charge on the gate capacitance). Once charged, it stays on with zero input power.*
8. *Silicon naturally forms a high-quality oxide (SiO₂) — an excellent insulator. Germanium oxide is water-soluble and mechanically poor.*